In [ ]:
import tensorflow as tf
import pickle
import gzip
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from fgnet import create_loss_svd, computeMetrics, normalizeFG
from fgnet import SimpleNet as PICnet
from bottle import *
sns.set()

In [ ]:
# produce dataset
# X bits as +1, -1

n = 4 # number of bits
M = 1500 # number of samples
batch_size = 50

# BSC error
p = 0.1

# produce random bits
X = 2*np.float32(np.random.randint(2,size=(M,n)))-1
Z = 2*(np.random.random(size=(M,n))<1-p)-1
Y = X*Z

# start session
session = tf.InteractiveSession()

# number of functions
d = 12
(x_input,f_output,keepX) = PICnet(n,name="Fnet",structure = [30, 30, 25, d])
(y_input,g_output,keepY) = PICnet(n,name="Gnet",structure = [30, 30, 25, d])

# create loss function for finding PICs
objective,prodGiFG = create_loss_svd(f_output,g_output)
step = tf.train.GradientDescentOptimizer(1e-2).minimize(objective)

# train
session.run(tf.global_variables_initializer())




In [ ]:
n_steps = 10000
obj = []
vals = []
F_vals = []
prod_vals = []
for k in tqdm(range(n_steps)):
    
    if k%500==0:
        obj.append(objective.eval(feed_dict= {x_input:X,y_input:Y}))
        prod_vals.append(prodGiFG.eval(feed_dict= {x_input:X,y_input:Y}))
        F_vals.append(f_output.eval(feed_dict= {x_input:X,y_input:Y}))
        vals.append(k)
    if np.isnan(obj[-1]):
            break
    session.run(step,feed_dict= {x_input:X,y_input:Y})
    
    
plt.plot(vals,d+1+np.array(obj))
plt.xlabel('Iteration')
plt.ylabel('Loss')


In [ ]:
F = f_output.eval(feed_dict= {x_input:X,y_input:Y})
G = g_output.eval(feed_dict= {x_input:X,y_input:Y})

A,a,B,b = normalizeFG(F,G)

In [ ]:
F2 = (F-a).dot(A)
G2 = (G-b).dot(B)
trueCorr = F2.transpose().dot(G2)/G.shape[0]
sns.heatmap(F2.transpose().dot(G2)/G.shape[0])
plt.title('Correlation between true F and G')
np.diag(trueCorr)


In [ ]:
# BSC error

M = 15000
# produce random bits
X2 = 2*np.float32(np.random.randint(2,size=(M,n)))-1
Z = 2*(np.random.random(size=(M,n))<1-p)-1
Y2 = X2*Z

F3 = f_output.eval(feed_dict= {x_input:X2,y_input:Y2})
G3 = g_output.eval(feed_dict= {x_input:X2,y_input:Y2})


F3 = (F3-a).dot(A)
G3 = (G3-b).dot(B)
trueCorr = F3.transpose().dot(G3)/G3.shape[0]
sns.heatmap(trueCorr)

plt.title('Correlation between true F and G')
np.diag(trueCorr)

In [ ]:
# plot f and g on same axis
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

# create strings of sequential bits
form = "{0:0"+str(n) +"b}"
listStrings  = [form.format(num) for num in range(2**n)]
Xref = np.array([[2*int(bit)-1 for  bit in num] for num in listStrings])

# compute ref outputs
Fref = f_output.eval(feed_dict= {x_input:Xref,y_input:Xref})
Gref = g_output.eval(feed_dict= {x_input:Xref,y_input:Xref})

# whiten F
wFref = (Fref-a).dot(A)
wGref = (Gref-b).dot(B)

ax.scatter(wFref[:,1],wFref[:,2],wFref[:,3],'o',alpha=.4)

# print text
for i in range(2**n):
    ax.text(wFref[i,1],wFref[i,2],wFref[i,3], listStrings[i], size=10, zorder=1, color='b',alpha=.8)

ax.scatter(wGref[:,1],wGref[:,2],wGref[:,3],'o',alpha=.4,color='r')
# print text
for i in range(2**n):
    ax.text(wGref[i,1],wGref[i,2],wGref[i,3], listStrings[i], size=10, zorder=1, color='r',alpha=.8)
    

    

In [ ]:
F2 = -F2
G2 = -G2
F2

In [ ]:
tf.reset_default_graph()
session.close()
session = tf.InteractiveSession()


# create bootle out framework


Wdim = 10

(x_inputB, outputB,keepProbB) = SimpleNet(n,name="bottlenet2",structure = [30, 30, 25, Wdim])

# normalize
(f_bottle_out,Pw) = bottleOut(outputB,name="bottleout")

# create g input
g_out = tf.placeholder(tf.float32,shape=[None,G2.shape[1]])

# create loss
(loss,barrier_val,Ifwy,Ifwx,g_approx) = create_loss_bottle(f_bottle_out,g_out,Pw,compression = 'MI', recovery = 'MI',clip_min = 1e-10,loss_coef = [4,2,10])

# create optimization step
step_size = tf.placeholder_with_default(1e-2,[])
step = tf.train.RMSPropOptimizer(step_size).minimize(loss)

session.run(tf.global_variables_initializer())
# train
n_steps = 5000
obj = []
Iy = []
Ix = []
vals = []
f_bottle = []
g_bottle = []
b=10;
s=1e-2
for k in tqdm(range(n_steps)):
    
    if k%2==0:
        obj.append(loss.eval(feed_dict= {x_inputB:X,g_out:G2,barrier_val:0.0}))
        Iy.append(Ifwy.eval(feed_dict= {x_inputB:X,g_out:G2}))
        Ix.append(Ifwx.eval(feed_dict= {x_inputB:X,g_out:G2}))
        f_bottle.append(f_bottle_out.eval(feed_dict= {x_inputB:X,g_out:G2}))
        g_bottle.append(g_approx.eval(feed_dict= {x_inputB:X,g_out:G2}))
        vals.append(k)
        if k>0 and np.isnan(obj[-1]) :
                break
    if k%500==0 and k>1:
        print(b)
        b=2*b
        s=s/1.5
    session.run(step,feed_dict= {x_inputB:X,g_out:G2,barrier_val:1/b,step_size:s})
    
    
plt.plot(vals,np.array(obj),label =  'obj')
plt.plot(vals,np.array(Iy), label = 'Iy')
plt.plot(vals,np.array(Ix), label = 'Ix')
plt.xlabel('Iteration')
plt.ylabel('Loss')                
plt.legend()
    